In [ ]:
import pandas as pd
import numpy as np
import quandl
import math
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style
import datetime
from statistics import mean
import random 
from sklearn import neighbors
from collections import Counter
import warnings

In [ ]:
df=quandl.get("EOD/MMM", authtoken="xcyzkuJxNDtzUpextG5H")
display(df)
df=df[['Open','High','Low','Close','Volume','Dividend','Split','Adj_Open','Adj_High','Adj_Low','Adj_Close','Adj_Volume']]
df['HL_PCT']=(df['Adj_High']-df['Adj_Close'])/df['Adj_Close']*100.0
df['PCT_change']=(df['Adj_Close']-df['Adj_Open'])/df['Adj_Open']*100.0
df=df[['Adj_Close','HL_PCT','PCT_change']]
forcast='Adj_Close'
df.fillna(-99,inplace=True)

forcastout=int(math.ceil(0.1*len(df)))#rounds up to integer value
df['label']=df[forcast].shift(-forcastout)


X=np.array(df.drop(['label'],1))#drop will return a new dataframe
#feature will be captal X, label will be lower case y
X=X[:-forcastout]
X=preprocessing.scale(X)
X_lat=X[-forcastout:]
display(X)
df.dropna(inplace=True)
y=np.array(df['label'])
y=np.array(df['label'])
display(y)
X_train,X_test,y_train,y_test=sklearn.model_selection.train_test_split(X,y,test_size=0.3)
clf=LinearRegression()
clf.fit(X_train,y_train)
accuracy=clf.score(X_test,y_test)
#display(accuracy,forcastout)
forcast_set=clf.predict(X_lat)
style.use=('ggplot')
df['Forcast']=np.nan
last_date=df.iloc[-1].name
last_unix=last_date.timestamp()
one_day=86400#how many seconds per day
next_unix=last_unix+one_day

for i in forcast_set:
    next_date=datetime.datetime.fromtimestamp(next_unix)
    next_unix+=one_day
    df.loc[next_date]=[np.nan for _ in range(len(df.columns)-1)]+[i]
df['Adj_Close'].plot()
df['Forcast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
print('accuracy: ',accuracy)
plt.savefig('stock1.jpg',dpi=1400)
df.to_csv('suu.csv')